In [1]:
import numpy as np
from hmmlearn.hmm import MultinomialHMM

np.random.seed(12345)
model = MultinomialHMM(n_components=3)
model.startprob_ = np.array([0.2, 0.4, 0.4])
model.transmat_ = np.array([[0.5, 0.2, 0.3],
                            [0.3, 0.5, 0.2],
                            [0.2, 0.3, 0.5]])
model.emissionprob_ = np.array([[0.5, 0.5], 
                                [0.4, 0.6], 
                                [0.7, 0.3]])
X, Z = model.sample(10)

In [2]:
class HiddenMarkovModel:
    
    def __init__(self,n_components=1,max_iter=100,\
                 pi=None,A=None,B=None):
        self.n_components = n_components
        self.max_iter = max_iter
        self.pi = pi
        self.A = A
        self.B = B
        
    def compute_alpha(self,X):
        T = len(X)
        alpha = np.zeros((T,self.n_components))
        for i in range(T):
            if i==0:
                alpha[i,:] = self.pi*self.B[:,X[i]].reshape(-1)
            else:
                alpha[i,:] = np.sum(self.A.T*alpha[i-1,:],1)*self.B[:,X[i]].reshape(-1)
        return alpha
    
    def compute_beta(self,X):
        T = len(X)
        beta = np.ones((T,self.n_components))
        for i in range(T-1,0,-1):
            beta[i-1,:] = np.sum(self.A*self.B[:,X[i]].reshape(1,-1)*beta[i,:],0)
        return beta   
    
    def compute_gamma(self):
        return self.alpha*self.beta/np.sum(self.alpha*self.beta,1).reshape(-1,1)
    
    def compute_xi(self,X):
        T,N = self.alpha.shape
        xi = np.zeros((T-1,N,N))
        for t in range(0,T-1):
            for i in range(0,N):
                for j in range(0,N):
                    xi[t,i,j] = self.alpha[t,i]*self.A[i,j]*self.B[j,X[t+1]]*self.beta[t+1,j]
        return xi/np.sum(np.sum(xi,2),1).reshape(-1,1,1)
        
    def fit(self,X):
        N,K,T = self.n_components,len(np.unique(X)),len(X)
        
        self.pi = np.array([np.random.random() for _ in range(N)])
        self.pi = self.pi/np.sum(self.pi)
        
        self.A = np.array([np.random.random() for _ in range(N*N)]).reshape(N,N)
        self.A = self.A/np.sum(self.A,1).reshape(-1,1)
        
        self.B = np.array([np.random.random() for _ in range(N*K)]).reshape(N,K)
        self.B = self.B/np.sum(self.B,1).reshape(-1,1)

        
        self.alpha = np.zeros((T,N))
        self.beta = np.ones((T,N))
        self.xi = np.zeros((T-1,N,N))
        
        for _ in range(self.max_iter):
            self.alpha = self.compute_alpha(X)
            self.beta = self.compute_beta(X)
            self.gamma = self.compute_gamma()
            self.xi = self.compute_xi(X)
            
            self.A = np.sum(self.xi,axis=0).reshape(N,N)/np.sum(self.gamma[:-1,:],axis=0)
            self.B = np.concatenate([(np.sum((X==i)*self.gamma,0)/np.sum(self.gamma,0)).\
                    reshape(N,1) for i in range(K)],1)
            self.pi = self.gamma[0,:].reshape(-1)
            
    def approx(self,O):
        alpha = self.compute_alpha(O)
        beta = self.compute_beta(O)
        gamma = alpha*beta/np.sum(alpha*beta,1).reshape(-1,1)
        return np.argmax(gamma,1)
    
    def viterbi(self,O):
        T,N = len(O),self.n_components
        delta = np.zeros((T,N))
        psi = np.zeros((T,N))
        for t in range(T):
            if t==0:
                delta[t,:] = self.pi*self.B[:,O[t]].reshape(-1)
            else:
                delta[t,:] = np.max(self.A*delta[t-1,:],1)*self.B[:,O[t]].reshape(-1)
                psi[t,:] = np.argmax(self.A*delta[t-1,:],1)
        path = []
        max_prob = np.max(delta[-1,:])
        path.append(np.argmax(delta,1)[-1])
        for t in range(T-1,0,-1):
            path.append(int(psi[t,int(path[-1])]))
        return np.array(path[::-1])     

In [3]:
clf = HiddenMarkovModel(n_components=3)
clf.pi = np.array([0.2, 0.4, 0.4])
clf.A = np.array([[0.5, 0.2, 0.3],
                  [0.3, 0.5, 0.2],
                  [0.2, 0.3, 0.5]])
clf.B = np.array([[0.5, 0.5], 
                  [0.4, 0.6], 
                  [0.7, 0.3]])

In [4]:
print('状态序列:',Z,'\n近似算法估计:',clf.approx(X),'\n维特比算法估计:',clf.viterbi(X))

状态序列: [2 0 1 2 2 2 0 0 2 2] 
近似算法估计: [1 1 1 1 1 1 1 1 1 1] 
维特比算法估计: [2 2 2 2 2 1 1 1 1 1]
